# 4 bit activation, 4 bit weight

In [11]:
import argparse
import os
import time
import shutil
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
batch_size = 128
model_name = "VGG16_quant_4b"
model = VGG16_quant_4b()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d_4b(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d_4b(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d_4b(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d_4b(
      128, 128, kernel_size=(3

In [12]:
# # train the VGG quant model
# print('GPU: ', use_gpu)
# print('=> Building model...')

# lr = 1e-3
# weight_decay = 1e-5
# epochs = 200
# best_prec = 0

# model.cuda()
# criterion = nn.CrossEntropyLoss().cuda()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# fdir = 'result/'+str(model_name)
# if not os.path.exists('result'):
#     os.makedirs('result')
# if not os.path.exists(fdir):
#     os.makedirs(fdir)
# else:
#     checkpoint = torch.load(fdir+ '/model_best.pth.tar')
#     model.load_state_dict(checkpoint['state_dict'])

# for epoch in range(0, epochs):
    
#     adjust_learning_rate(optimizer, epoch)

#     train(trainloader, model, criterion, optimizer, epoch)

#     # evaluate on test set
#     print("Validation starts")
#     prec = validate(testloader, model, criterion)

#     # remember best precision and save checkpoint
#     is_best = prec > best_prec
#     best_prec = max(prec,best_prec)
#     print('best acc: {:1f}'.format(best_prec))
#     save_checkpoint({
#         'epoch': epoch + 1,
#         'state_dict': model.state_dict(),
#         'best_prec': best_prec,
#         'optimizer': optimizer.state_dict(),
#     }, is_best, fdir)


In [13]:
PATH = "result/VGG16_quant_4b/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9242/10000 (92%)



In [14]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
j = 0
for i, layer in enumerate(model.features):
    if isinstance(layer, QuantConv2d_4b):
        layer.register_forward_pre_hook(save_output)
        print(f'{i}th layer prehooked at save_output index {j}')
        j += 1
####################################################

dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)

0th layer prehooked at save_output index 0
3th layer prehooked at save_output index 1
7th layer prehooked at save_output index 2
10th layer prehooked at save_output index 3
14th layer prehooked at save_output index 4
17th layer prehooked at save_output index 5
20th layer prehooked at save_output index 6
24th layer prehooked at save_output index 7
27th layer prehooked at save_output index 8
29th layer prehooked at save_output index 9
33th layer prehooked at save_output index 10
36th layer prehooked at save_output index 11
39th layer prehooked at save_output index 12


In [15]:
weight_q = model.features[27].weight_q
w_alpha = model.features[27].weight_quant.wgt_alpha
w_bit = 4

weight_int = weight_q / (w_alpha / (2**(w_bit-1)-1))
weight_int = weight_int

In [16]:
act = save_output.outputs[8][0]
act_alpha  = model.features[27].act_alpha
act_bit = 4
act_quant_fn = act_quantization(act_bit)

act_q = act_quant_fn(act, act_alpha)

act_int = act_q / (act_alpha / (2**act_bit-1))

In [17]:
psum_int = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1, bias=False).cuda()
psum_int.weight = torch.nn.parameter.Parameter(weight_int)
relu = model.features[28]

psum_int = psum_int(act_int)
psum_recovered = psum_int * (act_alpha / (2**act_bit-1)) * (w_alpha / (2**(w_bit-1)-1))
psum_recovered = relu(psum_recovered)

In [18]:
psum_ref = save_output.outputs[9][0]
difference = psum_recovered - psum_ref
print(difference.abs().sum())

tensor(0.0076, device='cuda:0', grad_fn=<SumBackward0>)


In [23]:
array_size = 8
ic = 8
oc = 8

X = act_int[0,:,:,:]
X = torch.reshape(X, (X.size(0), -1))
W = weight_int[0,:,:,:]
W = torch.reshape(W, (W.size(0), -1))
PSUM = psum_recovered[0,:,:,:]
PSUM = torch.reshape(PSUM, (PSUM.size(0), -1))
X.shape, W.shape, PSUM.shape

(torch.Size([8, 16]), torch.Size([8, 9]), torch.Size([8, 16]))

In [25]:
# activation file write
bit_precision = 4
file = open('4b_output/activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(round(X[7-j,i].item()))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    


In [26]:
# weight file write
bit_precision = 4
file = open('4b_output/weight.txt', 'w') #write to file
file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(W.size(1)): # time step (col)
    for j in range(W.size(0)): # row
        weight = round(W[7-j,i].item())
        if weight < 0:
            weight = 16 + weight
        W_bin = '{0:04b}'.format(weight)
        for k in range(bit_precision):
            file.write(W_bin[k])        
        file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    

In [28]:
# psum file write
bit_precision = 16
file = open('4b_output/psum.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(PSUM.size(1)): # time
    for j in range(PSUM.size(0)): # col
        psum_dec = round(PSUM[7-j,i].item())
        if psum_dec < 0:
            psum_dec = 2 ** 16 + psum_dec
        psum_bin = '{0:016b}'.format(psum_dec)
        for k in range(bit_precision):
            file.write(psum_bin[k])
        file.write(' ')
    file.write('\n')
file.close()